# Kikao cha 2 – Tathmini ya RAG kwa kutumia ragas

Tathmini mfumo wa msingi wa RAG ukitumia vipimo vya ragas: answer_relevancy, faithfulness, context_precision.


# Hali
Hali hii inatathmini mfumo mdogo wa Retrieval Augmented Generation (RAG) kwa njia ya ndani. Tunafanya yafuatayo:
- Kufafanua mkusanyiko mdogo wa hati za bandia.
- Kuweka hati na kutekeleza njia rahisi ya kupata usawa wa hati.
- Kuzalisha majibu yenye msingi wa ukweli kwa kutumia mfano wa ndani (Foundry Local / OpenAI-compatible).
- Kuhesabu vipimo vya ragas (`answer_relevancy`, `faithfulness`, `context_precision`).
- Kusaidia hali ya HARAKA (env `RAG_FAST=1`) ili kuhesabu tu umuhimu wa jibu kwa urahisi wa kurudia.

Tumia daftari hili kuthibitisha kuwa mfano wako wa ndani + mfumo wa embeddings unazalisha majibu yenye msingi wa ukweli kabla ya kupanua hadi mkusanyiko mkubwa wa hati.


### Maelezo: Usakinishaji wa Vitegemezi
Husakinisha maktaba zinazohitajika:
- `foundry-local-sdk` kwa usimamizi wa modeli za ndani.
- Kiolesura cha mteja `openai`.
- `sentence-transformers` kwa embeddings zenye msongamano.
- `ragas` + `datasets` kwa tathmini na hesabu ya vipimo.
- Kiongeza `langchain-openai` kwa kiolesura cha ragas LLM.

Ni salama kuendesha tena; ruka ikiwa mazingira tayari yameandaliwa.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### Maelezo: Uingizaji Muhimu & Vipimo
Inapakia maktaba za msingi na vipimo vya ragas. Vipengele muhimu:
- SentenceTransformer kwa embeddings.
- `evaluate` + vipimo vilivyochaguliwa vya ragas.
- `Dataset` kwa ajili ya kujenga mkusanyiko wa tathmini.
Uingizaji huu hauanzishi miito ya mbali (isipokuwa mzigo wa akiba ya modeli kwa embeddings).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### Maelezo: Toy Corpus & QA Ground Truth
Inafafanua mkusanyiko mdogo wa kumbukumbu ndani ya kumbukumbu (`DOCS`), seti ya maswali ya watumiaji, na majibu ya kweli yanayotarajiwa. Hii inaruhusu hesabu ya haraka na ya uhakika ya vipimo bila kupakua data kutoka nje. Katika hali halisi, ungechagua maswali ya uzalishaji pamoja na majibu yaliyopangwa kwa uangalifu.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### Maelezo: Kuanza Huduma, Embeddings & Kiraka cha Usalama
Inaanzisha meneja wa Foundry Local, inatumia kiraka cha usalama cha schema-drift kwa `promptTemplate`, inasuluhisha kitambulisho cha modeli, inaunda mteja anayelingana na OpenAI, na inahesabu mapema embeddings mnene kwa mkusanyiko wa hati. Hii inaweka hali inayoweza kutumika tena kwa urejeshaji + kizazi.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### Maelezo: Kazi ya Retriever
Inafafanua retriever rahisi wa kufanana kwa vekta kwa kutumia dot product juu ya embeddings zilizonormalishwa. Inarejesha hati za juu-k (k=2 kwa chaguo-msingi). Katika uzalishaji, badilisha na ANN index (FAISS, Chroma, Milvus) kwa kiwango na kasi.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### Maelezo: Kazi ya Uzalishaji
`generate` huunda mwongozo ulio na mipaka (mfumo unatoa maagizo ya kutumia KONTEXTI PEKEE) na kuita modeli ya ndani. Halijoto ya chini (0.1) inapendelea uchimbaji wa kweli zaidi ya ubunifu. Inarejesha maandishi ya jibu yaliyopunguzwa.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### Maelezo: Uanzishaji wa Mteja wa Akiba  
Inahakikisha `client` ipo hata kama seli ya uanzishaji ya awali ilirukwa au ilishindwa—inaepusha NameError wakati wa hatua za tathmini za baadaye.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### Maelezo: Mzunguko wa Tathmini & Vipimo
Huunda seti ya data ya tathmini (safu zinazohitajika: swali, jibu, muktadha, ukweli wa msingi, rejea) kisha inazunguka vipimo vilivyochaguliwa vya ragas.

Uboreshaji:
- FAST_MODE inazuia kwenye umuhimu wa jibu kwa majaribio ya haraka.
- Mzunguko wa kila kipimo huepuka upya kamili wa hesabu wakati kipimo kimoja kinashindwa.

Hutoa kamusi ya kipimo -> alama (NaN ikiwa kuna kushindwa).


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kwa usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
